In [1]:
import elasticsearch
import unicodedata
import csv
from elasticsearch_dsl.aggs import Terms, Nested
from elasticsearch_dsl.search import Search
from elasticsearch_dsl import Q
import networkx as nx
import pandas as pd
from pandas.io.json import json_normalize
import json
import pickle
import datetime
from pandasticsearch import Select
import numpy as np

es = elasticsearch.Elasticsearch(["149.165.170.209:9200"])

#####################################################################
# Make your Query with elasticsearch_dsl
#####################################################################
res = Search(using=es, index="packetbeat*").filter('range',**{"@timestamp":{'gte': '2018-11-22T00:00:00.000Z', 'lt' : '2018-11-22T23:59:59.999Z'}}).params(request_timeout=10000)#.sort("timestamp", {'order': "desc"})

#####################################################################
# Print the underlying JSON representation of your Query
#####################################################################
res_json = res.to_dict()
print(res_json)
    
#####################################################################
# Response execution
#####################################################################
response = res.execute()
#####################################################################

print("Filtered logs: %i" %res.count()) #There is an error on the number of matched logs
print("Total number of logs: %i" %response.hits.total)

{'query': {'bool': {'filter': [{'range': {'@timestamp': {'gte': '2018-11-22T00:00:00.000Z', 'lt': '2018-11-22T23:59:59.999Z'}}}]}}}
Filtered logs: 5443994
Total number of logs: 5443994


In [2]:
#####################################################################
# Make a small test for normalizing a json response (single flow) from elasticsearch
#####################################################################
test_json = {'flow_id': 'EQQA////DP//////FP8BAAH6Fj4hHCz6Fj5kj2kKAQFBCgEBGLQTxuw', 'host': {'name': 'elk-master'}, 'start_time': '2018-10-23T23:47:34.605Z', '@timestamp': '2018-10-23T23:48:20.193Z', 'type': 'flow', 'source': {'ip': '10.1.1.24', 'port': 60614, 'stats': {'net_bytes_total': 14750, 'net_packets_total': 48}, 'mac': 'fa:16:3e:64:8f:69'}, 'final': False, 'tags': ['beats_input_raw_event'], 'dest': {'ip': '10.1.1.65', 'port': 5044, 'stats': {'net_bytes_total': 2424, 'net_packets_total': 24}, 'mac': 'fa:16:3e:21:1c:2c'}, 'beat': {'hostname': 'elk-master', 'name': 'elk-master', 'version': '6.4.2'}, 'transport': 'tcp', '@version': '1', 'last_time': '2018-10-23T23:48:17.797Z'}
pandas_test_df = json_normalize(test_json)
pandas_test_df.head()
print(datetime.datetime.now())
# n_rows = len(pandas_df.index)
# if (n_rows % 1) == 0:
#     print(n_rows)


2019-07-19 22:43:35.629305


In [3]:
#print(res.scan()['hits']['hits'])

In [4]:
## # CNormalize each response (flow in response) and append to dataframe
#####################################################################
print("#####################")
print(datetime.datetime.now())
print("#####################")
counter = 0
gonzalo_list =[]

for hits in res.scan():
    #print(hits.meta.id)
    #print(hits.meta.index)
    temp_json = hits.to_dict()
    temp_json['meta_id'] = hits.meta.id
    temp_json['meta_index'] = hits.meta.index
    #temp_json_pretty = json.dumps(temp_json, indent=4)
    gonzalo_list.append(temp_json)
    #print(temp_json_pretty)
    #print("\n")
    counter+=1
    if (counter % 250000) == 0:
        print(datetime.datetime.now())
        print(counter)
        #print("\n")
print(datetime.datetime.now())
print(counter)
with open('/mnt/sdb/json_files/netflows_2018-11-22.json','w') as outfile:
    json.dump(gonzalo_list, outfile, ensure_ascii=False, indent=2)
#     temp_df = json_normalize(hits.to_dict())
#     #main_pandas_df = pd.concat([main_pandas_df,temp_df],axis=1)
#     main_pandas_df = main_pandas_df.append(temp_df, ignore_index=True, sort=False)
#     n_rows = len(main_pandas_df.index)
#     if (n_rows % 2500) == 0:
#         print(datetime.datetime.now())
#         print(n_rows)
#         print("\n")
    
#####################################################################
# Visualize the results
#####################################################################
#main_pandas_df.head(50)
outfile.close()

#####################
2019-07-19 22:43:35.687728
#####################
2019-07-19 22:44:58.152734
250000
2019-07-19 22:46:30.041573
500000
2019-07-19 22:48:08.701935
750000
2019-07-19 22:49:47.696667
1000000
2019-07-19 22:51:17.326489
1250000
2019-07-19 22:52:49.579998
1500000
2019-07-19 22:54:34.264381
1750000
2019-07-19 22:56:05.628415
2000000
2019-07-19 22:57:40.431123
2250000
2019-07-19 22:59:12.961027
2500000
2019-07-19 23:00:38.611544
2750000
2019-07-19 23:02:19.011273
3000000
2019-07-19 23:03:42.761540
3250000
2019-07-19 23:05:12.646972
3500000
2019-07-19 23:06:43.289765
3750000
2019-07-19 23:08:30.344765
4000000
2019-07-19 23:09:48.360063
4250000
2019-07-19 23:11:13.757775
4500000
2019-07-19 23:12:43.966241
4750000
2019-07-19 23:14:10.493545
5000000
2019-07-19 23:15:22.127119
5250000
2019-07-19 23:16:54.191934
5443994


In [5]:
print("#########################")
print("Done")
print("#########################")
print(datetime.datetime.now())

#########################
Done
#########################
2019-07-19 23:23:48.111208
